In [9]:
import xarray as xr
import glob
import numpy as np
import cftime

np.seterr(all='ignore');

In [10]:
fnames = sorted(glob.glob(f'../data/ecefiles/cmip6/thetao*.nc'))

nyears = len(fnames)

basin = ['EAIS','ROSS','AMUN','WEDD','PENS']

In [11]:
#Get time-independent variables
ds = xr.open_dataset('../data/ecefiles/areas.nc')
area = ds['O1t0.srf'].values;
ds.close()

ds = xr.open_dataset('../data/ecefiles/pi03/pi03_1m_18500101_18501231_grid_T.nc')
lat = ds['nav_lat'].values
lon = ds['nav_lon'].values
levmid = ds['olevel'].values
lev = ds['olevel_bounds'].values
time_bnds = ds['time_centered_bounds']
thick = ds['e3t'].values #Quasi-time-independent, treated as fixed
ds.close()
secs = (time_bnds[:,1]-time_bnds[:,0]).values / np.timedelta64(1, 's')

In [12]:
#Get weighted mask for basin averages
lons = np.repeat(lon[np.newaxis,:,:],len(levmid),axis=0)
lats = np.repeat(lat[np.newaxis,:,:],len(levmid),axis=0)
mask = np.repeat(np.zeros(lons.shape)[np.newaxis,:,:,:],len(basin),axis=0)
aweight = np.zeros(mask.shape)

for b,bb in enumerate(basin):
    mm = np.zeros(lons.shape)
    if b==0:
        #EAIS
        mm[((lons<173) & (lons>-10)) & (lats<-65) & (lats>-76)] = 1
        depp = 369
    if b==1:
        #ROSS
        mm[((lons>150) | (lons<-150)) & (lats<-76)] = 1
        depp = 312        
    if b==2:
        #AMUN
        mm[(lons>-150) & (lons<-80) & (lats<-70)] = 1
        depp = 305
    if b==3:
        #WEDD
        mm[(lons>-65) & (lons<-10) & (lats<-72)] = 1
        depp = 420
    if b==4:
        #APEN
        mm[(lons>-66) & (lons<-56) & (lats>-70) & (lats<-65)] = 1
        mm[(lons>-80) & (lons<-65) & (lats>-75) & (lats<-70)] = 1
        depp = 420
        
    z0 = depp-50.
    i0 = np.argmax(lev[:,1]>z0)
    mm[:i0,:,:] = 0
    w0 = (lev[i0,1]-z0)/(lev[i0,1]-lev[i0,0])
    mm[i0,:,:] = w0*mm[i0,:,:]
    for j in range(0,lon.shape[0]):
        for i in range(0,lon.shape[1]):
            if np.nansum(thick[0,i0:,j,i]) == 0:
                continue
            z1 = depp+50.
            i1 = np.argmin(lev[:,1]<z1)
            w1 = (z1-lev[i1,0])/(lev[i1,1]-lev[i1,0])
            mm[i1,j,i] = w1*mm[i1,j,i]
            mm[i1+1:,j,i] = 0
    mask[b,:,:,:] = mm*np.where(np.isnan(thick[0,:,:,:]),0,thick[0,:,:,:])/100.
    aweight[b,:,:,:] = mask[b,:,:,:]*area[np.newaxis,:,:] 

In [13]:
#Calculate basin-average annual time series
tbas = np.zeros((nyears,len(basin)))
ttime = np.arange(nyears)
months = np.arange(0,12)

c = 0
for f,fname in enumerate(fnames):
    ds = xr.open_dataset(fname,use_cftime=True)
    time = ds['time'].values
    temp = ds['thetao'].values
    ds.close()
    
    year0 = int(fname[-16:-12])
    
    tb = np.zeros((len(basin)))
    ny = int(len(time)/12)
    for y in np.arange(0,ny):
        for b,bas in enumerate(basin):
            for m,mm in enumerate(months):
                tbb = np.nansum(temp[m+12*y,:,:,:]*aweight[b,:,:,:])/np.nansum(aweight[b,:,:,:])
                tb[b] += tbb*secs[m]
            tbas[c,b] = tb[b]/sum(secs)
        print(year0+y,c,tbas[c,:])
        tb = np.zeros((len(basin)))
        c += 1

2561 0 [ 0.13691192  0.40941645  2.12253228 -1.35491744  0.10217831]
2562 1 [ 0.12577545  0.11447443  1.97615591 -1.37769864  0.03166873]
2563 2 [ 0.13963115 -0.20742492  1.97727642 -1.47451662  0.00998558]
2564 3 [ 0.19841492 -0.28593784  2.05556587 -1.37035259 -0.1001873 ]
2565 4 [ 0.27600704 -0.24048076  2.0642275  -1.24724473 -0.25795895]
2566 5 [ 0.29935772 -0.20569359  1.97299307 -1.16465865 -0.37360296]
2567 6 [ 0.26730489 -0.22772812  1.88965773 -1.28556323 -0.34798385]
2568 7 [ 0.2670477  -0.10924946  2.00304844 -1.31864969 -0.24173888]
2569 8 [ 0.20482057  0.3700926   2.07282821 -1.32999288 -0.05138732]
2570 9 [ 0.15368575  0.53224878  2.14252865 -1.37778947  0.08160538]
2571 10 [ 0.19825576  0.18232694  2.12604029 -1.3908014   0.09937251]
2572 11 [ 0.21889287  0.14639836  2.18941088 -1.33065347  0.01428978]
2573 12 [ 0.24178095  0.54165916  2.23840431 -1.31657952 -0.14847538]
2574 13 [ 0.25059801  0.88274948  2.20478576 -1.30460478 -0.03631803]
2575 14 [ 0.24935171  0.855249

In [15]:
temp2 = xr.DataArray(tbas,dims=('time','basin'),coords={'time':ttime-nyears,'basin':basin},attrs={'unit':'degrees Celcius','long_name':'temperature time series per basin'})

ds = xr.Dataset({'temp':temp2})
ds.to_netcdf(f'../data/temperature_cmip6.nc')
ds.close()